<a href="https://colab.research.google.com/github/jatinmishra/Text-Summarizer/blob/main/BlogSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 2.1MB 7.5MB/s 
     |████████████████████████████████| 3.2MB 38.6MB/s 
     |████████████████████████████████| 890kB 41.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=48929429f63c117d3a6f3838c7a4b74410456e740fea0f73b202b93dc7bab036
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

In [3]:
summarizer = pipeline("summarization")

/usr/local/lib/python3.7/dist-packages/transformers/models/bart/configuration_bart.py:178: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions.The config can simply be saved and uploaded again to be fixed.
  f"Please make sure the config includes `forced_bos_token_id={self.bos_token_id}` in future versions."


In [18]:
#URL = "https://towardsdatascience.com/a-bayesian-take-on-model-regularization-9356116b6457"
#URL = "https://hackernoon.com/will-the-game-stop-with-gamestop-or-is-this-just-the-beginning-2j1x32aa"
URL = "https://medium.com/dev-genius/the-curious-case-of-code-reviews-6a1ce47c8b1f"

In [19]:
r = requests.get(URL)

In [20]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [21]:
max_chunk = 500

In [22]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [23]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [26]:
len(chunks)

2

In [25]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

In [27]:
res[0]

{'summary_text': ' There are two sides to this “very expensive” coin: Reviewing code and getting your code reviewed . The key point in code review is effective communication . Both the persons should ensure that they provide clear and valid explanations .'}

In [28]:
len(res)

2

In [29]:
' '.join([summ['summary_text'] for summ in res])

' There are two sides to this “very expensive” coin: Reviewing code and getting your code reviewed . The key point in code review is effective communication . Both the persons should ensure that they provide clear and valid explanations .  Follow the writers, publications, and topics that matter to you, and you’ll see them on your homepage and in your inbox . If you have a story to tell, knowledge to share, or a perspective to offer — welcome home . It’s easy and free to post your thinking on any topic .'

In [30]:
text = ' '.join([summ['summary_text'] for summ in res])

In [31]:
text

' There are two sides to this “very expensive” coin: Reviewing code and getting your code reviewed . The key point in code review is effective communication . Both the persons should ensure that they provide clear and valid explanations .  Follow the writers, publications, and topics that matter to you, and you’ll see them on your homepage and in your inbox . If you have a story to tell, knowledge to share, or a perspective to offer — welcome home . It’s easy and free to post your thinking on any topic .'